In [2]:
import pymongo, requests
import pandas as pd

In [13]:
#client
ip='13.125.255.151'
client = pymongo.MongoClient("mongodb://{ip}:27017".format(ip=ip))
client

MongoClient(host=['13.125.255.151:27017'], document_class=dict, tz_aware=False, connect=True)

In [16]:
# dss 데이터 베이스 선택
db = client.dss
db

Database(MongoClient(host=['13.125.255.151:27017'], document_class=dict, tz_aware=False, connect=True), 'dss')

In [17]:
# 데이터 베이스의 컬렉션 리스트 확인
db.collection_names()

['info']

In [18]:
# info 컬렉션 선택 
collection = db.info
collection

Collection(Database(MongoClient(host=['13.125.255.151:27017'], document_class=dict, tz_aware=False, connect=True), 'dss'), 'info')

### find

In [24]:
# find_one : 한개의 document 가져옴
document = collection.find_one({"subject":"sass"})
type(document), document

(dict,
 {'_id': ObjectId('5b5156bef159f19b01557e5f'),
  'comments': [{'msg': 'hello', 'name': 'alice'}],
  'level': 4.0,
  'subject': 'sass'})

In [25]:
# find : 여러개의 document 가져옴
documents = collection.find()
type(document), documents

(dict, <pymongo.cursor.Cursor at 0x1172ab780>)

In [26]:
datas = list(documents)
len(datas)

15

In [27]:
list(documents)

[]

In [29]:
# count
documents = collection.find()
documents.count()

15

In [31]:
# query문을 추가해서 find, sort
# 레벨이 3이상인 document를 내림차순으로 정렬
documents = collection.find({"level":{"$lte":3}}).sort("level", pymongo.DESCENDING)
list(documents)

[{'_id': ObjectId('5b5156bef159f19b01557e5c'),
  'comments': [{'msg': 'check', 'name': 'po'},
   {'msg': 'java is good!', 'name': 'peter'},
   {'msg': 'java is good!', 'name': 'peter'},
   {'msg': 'css is good!', 'name': 'alice'},
   {'msg': 'java is good!', 'name': 'peter'},
   {'msg': 'css is good!', 'name': 'alice'}],
  'level': 3.0,
  'subject': 'java'},
 {'_id': ObjectId('5b5156bef159f19b01557e5d'),
  'comments': [{'msg': 'nice', 'name': 'jin'}],
  'level': 3.0,
  'subject': 'javascript'},
 {'_id': ObjectId('5b591f76e677a829df1c7cb9'),
  'comments': [{'msg': 'nice', 'name': 'jin'}],
  'level': 3.0,
  'subject': 'javascript'},
 {'_id': ObjectId('5b5156bef159f19b01557e5b'),
  'comments': [{'msg': 'bad', 'name': 'jin'}, {'msg': 'hi', 'name': 'alice'}],
  'level': 1.0,
  'subject': 'r'},
 {'_id': ObjectId('5b5156bef159f19b01557e5e'),
  'comments': [],
  'level': 1.0,
  'subject': 'html'},
 {'_id': ObjectId('5b591f76e677a829df1c7cb7'),
  'comments': [{'msg': 'bad', 'name': 'jin'}, {'ms

### insert

In [34]:
# 한개의 데이터 저장
data = {"subject":"css", 
        "level":1, 
        "comments":[{"name":"peter","msg":"easy"}]
       }
result = collection.insert_one(data)
result

In [35]:
result.inserted_id

ObjectId('5b592eb845c935767b88faa8')

In [38]:
# 여러개의 데이터 저장
datas = [
        {"subject":"webpack", "level":4, "comments":[{"name":"peter","msg":"hard"}]},
        {"subject":"html", "level":1, "comments":[{"name":"peter","msg":"easy"}]},
        {"subject":"gulp", "level":2, "comments":[{"name":"peter","msg":"normal"}]},
       ]
result = collection.insert_many(datas)
result

In [39]:
result.inserted_ids

[ObjectId('5b592fa445c935767b88faa9'),
 ObjectId('5b592fa445c935767b88faaa'),
 ObjectId('5b592fa445c935767b88faab')]

## 직방 API를 이용하여 데이터를 크롤링한 후에 바로 mongoDB에 저장
- 직방 api url을 알아내기
- requests를 이용하여 api url로 json 포멧의 데이터를 가져오기
- json 데이터 포맷을 우리가 원하는 데이터 형태로 parsing -> 리스트(딕셔너리1, 딕셔너리2, ...)
- insert_many를 이용하여 mongoDB 데이터로 저장

In [64]:
url = "https://api.zigbang.com/v3/items?detail=true&item_ids=[12186936,12148806,12232092,12047734,12256559,12106966,12033155,12211322,12209347,12248807,12187776,12265413,12149873,12264667,12212136,12248423,12184621,12243706,12151201,12242579,12253248,12194520,12214149,12178296,12220144,12083330,11867881,12268533,12247061,12154439,12050466,12218781,12227475,12253341,12261568,11999697,12199634,12258871,12232582,12000486,11756839,12203399,12239829,12248400,12168471,12162133,11770488,12192341,12165908,12195624,12200620,12095467,12226491,12241377,12241213,12170521,12039279,12199080,11719700,12094974]"
response = requests.get(url)
response

<Response [200]>

In [65]:
zigbang_dict_list = response.json().get("items")
len(zigbang_dict_list)

60

In [66]:
zigbang_dict_list[0]

{'header': False,
 'header_height': 0,
 'item': {'_floor': '4층',
  'address1': '서울시 송파구 송파1동',
  'address2': '132',
  'address3': None,
  'agent_address1': '서울특별시 송파구 백제고분로39길 12, 서희빌딩 103호',
  'agent_comment': '저희 금탑공인중개사는 허위매물없이 실매물로만 손님 여러분께 최선을 다해 보답해 드리겠습니다. \n회원업소간 공동네트웍으로 one stop 서비스를 실시간 제공하며, 다년간의 노하우로 차별화된 중개와 맞춤 중개를 약속하겠습니다.\n언제든지 문의주세요^^\n\n카톡ID:  minta06',
  'agent_email': 'latefall--@hanmail.net',
  'agent_lat': 37.50547790527341,
  'agent_lng': 127.104957580566,
  'agent_local1': '서울시',
  'agent_local2': '송파구',
  'agent_mobile': '0507-1281-9400',
  'agent_name': '금탑공인중개사(임창미)',
  'agent_no': 24976,
  'agent_phone': '02-418-9877',
  'bjd_code': '',
  'bonbun_code': '132',
  'bubun_code': '',
  'building': None,
  'building_type': '다세대건물',
  'contract': '서울특별시',
  'deposit': 22000,
  'description': '♥석촌역 도보거리. 9호선개통시 도보 2분이내거리.\n\n♥세입자 있는 관계로 공실사진으로 대체합니다.\n\n♥코너집이라 채광좋아요\n\n♥신축급 정말 깔끔한집.\n\n♥주차가능합니다.\n\n♥전세자금대출가능합니다.\n\n♥CCTV등 보안상태 최상급입니다.\n\n♥거실 TV, 쇼파배치 가능한 반듯한 구조입니다.\

In [67]:
item_list = [item['item'] for item in zigbang_dict_list]
len(item_list)

60

In [68]:
item_list

[{'_floor': '4층',
  'address1': '서울시 송파구 송파1동',
  'address2': '132',
  'address3': None,
  'agent_address1': '서울특별시 송파구 백제고분로39길 12, 서희빌딩 103호',
  'agent_comment': '저희 금탑공인중개사는 허위매물없이 실매물로만 손님 여러분께 최선을 다해 보답해 드리겠습니다. \n회원업소간 공동네트웍으로 one stop 서비스를 실시간 제공하며, 다년간의 노하우로 차별화된 중개와 맞춤 중개를 약속하겠습니다.\n언제든지 문의주세요^^\n\n카톡ID:  minta06',
  'agent_email': 'latefall--@hanmail.net',
  'agent_lat': 37.50547790527341,
  'agent_lng': 127.104957580566,
  'agent_local1': '서울시',
  'agent_local2': '송파구',
  'agent_mobile': '0507-1281-9400',
  'agent_name': '금탑공인중개사(임창미)',
  'agent_no': 24976,
  'agent_phone': '02-418-9877',
  'bjd_code': '',
  'bonbun_code': '132',
  'bubun_code': '',
  'building': None,
  'building_type': '다세대건물',
  'contract': '서울특별시',
  'deposit': 22000,
  'description': '♥석촌역 도보거리. 9호선개통시 도보 2분이내거리.\n\n♥세입자 있는 관계로 공실사진으로 대체합니다.\n\n♥코너집이라 채광좋아요\n\n♥신축급 정말 깔끔한집.\n\n♥주차가능합니다.\n\n♥전세자금대출가능합니다.\n\n♥CCTV등 보안상태 최상급입니다.\n\n♥거실 TV, 쇼파배치 가능한 반듯한 구조입니다.\n\n♥신혼부부, 1인세대 살기 좋은구조입니다.',
  'description_og'

In [89]:
zigbang = client.crawling.zigbang
zigbang

Collection(Database(MongoClient(host=['13.125.255.151:27017'], document_class=dict, tz_aware=False, connect=True), 'crawling'), 'zigbang')

In [90]:
zigbang.insert_many(item_list)

In [91]:
# 렌트 비용이 50만원 이하의 데이터를 추출
QUERY = {"rent":{"$lte":50}}
documents = zigbang.find(QUERY)
datas = list(documents)
len(datas)

45

In [83]:
# pandas의 데이터 프레임으로 만들기
df = pd.DataFrame(datas)
df

,_floor,_id,address1,address2,address3,agent_address1,agent_comment,agent_email,agent_lat,agent_lng,...,updated_at2,user_email,user_has_no_penalty,user_has_penalty,user_intro,user_mobile,user_name,user_no,user_phone,view_count
0,4층,5b5934e645c935767b88fae8,서울시 송파구 송파1동,132,None,"서울특별시 송파구 백제고분로39길 12, 서희빌딩 103호",저희 금탑공인중개사는 허위매물없이 실매물로만 손님 여러분께 최선을 다해 보답해 드리...,latefall--@hanmail.net,37.505478,127.104958,...,2018-07-18,min_ta@naver.com,True,False,,0507-1282-6549,중개보조원 민병권,227750,0507-1282-6549,265
1,2층,5b5934e645c935767b88fae9,서울시 송파구 가락동,13-18,None,서울특별시 송파구 송이로23길 14,저희 미소부동산은 경찰병원역 3번출구 바로 앞에 있습니다. \n\n가락시장역⇔오금역...,dlaudgus0747@naver.com,37.495532,127.124853,...,2018-07-23,dlaudgus0747@naver.com,True,False,경찰병원역 3번 출구 앞에 위치한 미소부동산입니다 \n가락시장역에서 오금역까지 3...,0507-1281-1573,대표공인중개사 이명현,4509136,0507-1281-1573,223
2,4층,5b5934e645c935767b88faeb,서울시 송파구 가락본동,71-3,None,서울특별시 송파구 가락동 71-3,,ybk49@hanmail.net,37.497353,127.119667,...,2018-07-02,ybk49@hanmail.net,True,False,,0507-1282-1287,대표공인중개사 윤병관,1332859,0507-1282-1287,1431
3,3층,5b5934e645c935767b88faec,서울시 송파구 가락동,169-1,None,서울특별시 송파구 송파동 84-4,,d4200008.dasan4200008@naver.com,37.505547,127.108475,...,2018-07-24,bedmall@naver.com,True,False,,0507-1280-2324,중개보조원 박정주,772171,0507-1280-2324,30
4,4층,5b5934e645c935767b88faed,서울시 송파구 송파동,109-10,None,서울특별시 송파구 백제고분로42길 17 101호,,bank420@hanmail.net,37.505215,127.110489,...,2018-07-07,bank420@hanmail.net,True,False,,0507-1280-4869,대표공인중개사 강성룡,481700,0507-1280-4869,500
5,2층,5b5934e645c935767b88faee,서울시 송파구 오금동,82-7,None,서울특별시 송파구 백제고분로36가길 9,카톡ID : ducks57 상담 환영♥\n\n\n강동구 송파구 매물 다량 보유하고있...,woori7443@naver.com,37.504227,127.101097,...,2018-07-21,onnuri0999@naver.com,True,False,"안녕하세요, 온누리공인중개사 사무소입니다.\n\n계약에서 입주까지 안전한 거래로 책...",0507-1282-0069,중개보조원 최규덕,1322512,0507-1282-0069,542
6,반지하,5b5934e645c935767b88faef,서울시 송파구 송파동,112-5,None,서울특별시 송파구 백제고분로36가길 17,✅ 해당 매물 이외에도 다수의 매물 보유중 입니다.\n\n✅ 부담없이 문의 주세요....,gil3379@naver.com,37.504448,127.101997,...,2018-07-19,gil3379@naver.com,True,False,,0507-1282-4616,소속공인중개사 권성우,2862823,0507-1282-4616,142
7,3층,5b5934e645c935767b88faf0,서울시 송파구 방이동,187-16,None,서울특별시 송파구 방이동 135-15 1층 101호,허위 매물 없이 성실하게 소개해드립니다.\n\n언제나 친절한 상담을 약속합니다.\n...,junnooo@naver.com,37.510414,127.118599,...,2018-07-18,junnooo@naver.com,True,False,언제나 친절한 상담을 약속합니다.\n\n송파 전지역 매물을 전문으로 취급하고 있습니...,0507-1280-1901,대표공인중개사 이준호,2201186,0507-1280-1901,77
8,2층,5b5934e645c935767b88faf1,서울시 송파구 방이동,153-11,None,서울특별시 송파구 백제고분로48길 3 1층,💖 15년동안 송파구에서 영업해온 베테랑 부동산 입니다. \n\n💖공동중개망을 통해...,50011kim@hanmail.net,37.511837,127.113762,...,2018-07-23,50011kim@hanmail.net,True,False,,0507-1281-6347,소속공인중개사 강경화,33149,0507-1281-6347,22
9,5층,5b5934e645c935767b88faf2,서울시 송파구 가락동,174-24,None,"서울특별시 송파구 동남로8길 31 ,1층",자세한 내용은 연락 주시면 \n\n친절히 상담해드리겠습니다^^ \n\n최선을 다하겠...,hyl0822@naver.com,37.488857,127.127449,...,2018-07-23,hyl0822@naver.com,True,False,,0507-1281-8958,대표공인중개사 한유림,3470741,0507-1281-8958,254


In [86]:
df[["address1","rent","user_name"]].tail(10)

,address1,rent,user_name
35,서울시 송파구 송파동,0,중개보조원 이세리
36,서울시 송파구 방이동,50,중개보조원 김현민
37,서울시 강동구 성내동,0,대표공인중개사 이성두
38,서울시 송파구 방이동,0,대표공인중개사 김태운
39,서울시 송파구 방이동,0,대표공인중개사 김국천
40,서울시 송파구 오금동,0,대표공인중개사 이지윤
41,서울시 송파구 송파동,0,소속공인중개사 우정구
42,서울시 송파구 방이동,35,중개보조원 홍세림
43,서울시 송파구 방이동,45,대표공인중개사 백승원
44,서울시 송파구 송파동,0,대표공인중개사 임창미


In [87]:
# drop - collection
client.crawling.drop_collection("zigbang")

{'nIndexesWas': 1, 'ns': 'crawling.zigbang', 'ok': 1.0}

In [88]:
# drop - database
client.drop_database("crawling")

#### Quiz

- deposit이 3000이하의 데이터를 "agent_mobile","deposit","rent" 컬럼이 출력되도록 dataframe을 만들어라.

In [106]:
# 렌트 비용이 50만원 이하의 데이터를 추출
documents = zigbang.find(
                            {"deposit":{"$lte":3000}}, # where
                            {"_id":False, "agent_mobile":True, "deposit":True, "rent":True} # select
                        )
datas = list(documents)
len(datas)

27

In [111]:
pd.DataFrame(datas).tail(10)

,agent_mobile,deposit,rent
17,0507-1280-4869,2000,70
18,0507-1281-7400,1500,45
19,0507-1280-1901,1000,60
20,0507-1280-1032,100,110
21,0507-1282-1097,1000,50
22,0507-1282-8670,150,98
23,0507-1281-7400,1000,65
24,0507-1282-3373,500,55
25,0507-1281-7400,2000,35
26,0507-1281-7596,1000,45
